In [1]:
import pandas as pd
import pymongo
import json
import datetime as dt

In [2]:
# import all cvs from data folder
earthquake_csv = pd.read_csv("../Earthquake_data_cleaned.csv")
floods_csv = pd.read_csv("../floods_data.csv")
tornado_csv = pd.read_csv("../tor_output.csv")
hurricane_csv = pd.read_csv("../hurricane_cleaned_df.csv")

/Users/jeremyelam/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# drop unnecessary column 
earthquake_csv = earthquake_csv.drop(columns = ["Unnamed: 0"])

In [4]:
earthquake_df = pd.DataFrame(earthquake_csv)
floods_df = pd.DataFrame(floods_csv)
hurricane_df = pd.DataFrame(hurricane_csv)
tornado_df = pd.DataFrame(tornado_csv)

## Data Cleaning for Earthquake Data

In [5]:
earthquake_df["Date"] = earthquake_df.Year.map(str)+'-'+ earthquake_df.Month.map(str) + '-' + earthquake_df.Day.map(str)


In [6]:
earthquake_df.Date = pd.to_datetime(earthquake_df.Date)

In [7]:
earthquake_df = earthquake_df.dropna()

In [8]:
earthquake_df.Hour = earthquake_df.Hour.astype(int)

In [9]:
earthquake_df.Minute = earthquake_df.Minute.astype(int)

In [10]:
earthquake_df.Minute = earthquake_df.Minute.map("{:02}".format)  
earthquake_df.Hour  = earthquake_df.Hour.map("{:02}".format)

In [11]:
earthquake_df["Time"] = earthquake_df.Hour.map(str) + ":" + earthquake_df.Minute

In [12]:
earthquake_df = earthquake_df.rename(columns = {"Latitude": "latitude", "Longitude": "longitude"})

In [13]:
earthquake_df.Time = pd.to_datetime(earthquake_df.Time).dt.time

## DATA CLEANING FOR HURRICANE DATA

In [14]:
hurricane_df = hurricane_df.drop(columns = ["Unnamed: 0"])

In [15]:
hurricane_df[["Date","Time", "Time_Zone"]] = hurricane_df.iso_time.str.split(" ", expand = True)

In [16]:
hurricane_df.Date = pd.to_datetime(hurricane_df.Date)
hurricane_df.Time = pd.to_datetime(hurricane_df.Time).dt.time

## FLOOD DATA CLEANING

In [17]:
floods_df[["Date", "Time"]] = floods_df.Start_Time.str.split(" ", expand = True)

In [18]:
floods_df.Date = pd.to_datetime(floods_df.Date)
floods_df.Time = pd.to_datetime(floods_df.Time).dt.time

In [19]:
floods_df = floods_df.rename(columns = {"event_latitude": "latitude", "event_longitude": "longitude"})


In [20]:
floods_df = floods_df.drop(columns = ["Start_Date", "End_Time", "End_Date","End_Date","Start_Time"])

In [21]:
floods_df.event_begin_time = pd.to_datetime(floods_df.event_begin_time)
floods_df.event_end_time = pd.to_datetime(floods_df.event_end_time)


## TORNADO DATA CLEANING

In [22]:
tornado_df.Date = pd.to_datetime(tornado_df.Date, errors = 'coerce')
tornado_df.Time = pd.to_datetime(tornado_df.Time, errors = 'coerce')


In [25]:
tornado_df.Property_loss = tornado_df.Property_loss.replace({'K': '*1e3', 'M': '*1e6'}, regex=True).map(pd.eval).astype(int)

In [26]:
tornado_df=tornado_df.dropna()

## Change all column 

In [29]:
floods_df.columns = map(str.lower, floods_df.columns)
earthquake_df.columns = map(str.lower, earthquake_df.columns)
hurricane_df.columns = map(str.lower, hurricane_df.columns)
tornado_df.columns = map(str.lower, tornado_df.columns)

In [30]:
floods_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34586 entries, 0 to 34585
Data columns (total 15 columns):
episode_id          34586 non-null int64
state               34586 non-null object
event_type          34586 non-null object
event_begin_time    34586 non-null datetime64[ns]
event_end_time      34586 non-null datetime64[ns]
event_timezone      34586 non-null object
damage_property     34586 non-null float64
latitude            34586 non-null float64
longitude           34586 non-null float64
cz_type             34586 non-null object
cz_fips_code        34586 non-null int64
damage_crops        34586 non-null float64
source              34586 non-null object
date                34586 non-null datetime64[ns]
time                34586 non-null object
dtypes: datetime64[ns](3), float64(4), int64(2), object(6)
memory usage: 4.0+ MB


In [31]:
earthquake_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 193 entries, 0 to 193
Data columns (total 10 columns):
year         193 non-null int64
month        193 non-null int64
day          193 non-null int64
hour         193 non-null object
minute       193 non-null object
location     193 non-null object
latitude     193 non-null float64
longitude    193 non-null float64
date         193 non-null datetime64[ns]
time         193 non-null object
dtypes: datetime64[ns](1), float64(2), int64(3), object(4)
memory usage: 16.6+ KB


In [32]:
hurricane_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57095 entries, 0 to 57094
Data columns (total 14 columns):
index           57095 non-null int64
season          57095 non-null int64
basin           57095 non-null object
subbasin        57095 non-null object
name            57095 non-null object
iso_time        57095 non-null object
nature          57095 non-null object
latitude        57095 non-null float64
longitude       57095 non-null float64
usa_wind        57095 non-null float64
usa_pressure    30937 non-null float64
date            57095 non-null datetime64[ns]
time            57095 non-null object
time_zone       57095 non-null object
dtypes: datetime64[ns](1), float64(4), int64(2), object(7)
memory usage: 6.1+ MB


In [33]:
tornado_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61733 entries, 0 to 66140
Data columns (total 13 columns):
date             61733 non-null datetime64[ns]
time             61733 non-null datetime64[ns]
state            61733 non-null object
fscale           61733 non-null object
injured          61733 non-null float64
fatalities       61733 non-null float64
property_loss    61733 non-null int64
slat             61733 non-null float64
slon             61733 non-null float64
elat             61733 non-null float64
elon             61733 non-null float64
length           61733 non-null float64
width            61733 non-null float64
dtypes: datetime64[ns](2), float64(8), int64(1), object(2)
memory usage: 6.6+ MB


In [34]:
hurricane_data = json.loads(hurricane_df.T.to_json()).values()
floods_data = json.loads(floods_df.T.to_json()).values()
earthquake_data = json.loads(earthquake_df.T.to_json()).values()
tornado_data = json.loads(tornado_df.T.to_json()).values()

## NO SQL

In [35]:
conn = 'mongodb://localhost:27017'

client = pymongo.MongoClient(conn)

db = client.weather_data

db.hurricane_data.drop()

In [36]:
db.hurricane_data.insert(hurricane_data)

/Users/jeremyelam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  """Entry point for launching an IPython kernel.


[ObjectId('5e1a1e6464b3c70d0ea1c7a0'),
 ObjectId('5e1a1e6464b3c70d0ea1c7a1'),
 ObjectId('5e1a1e6464b3c70d0ea1c7a2'),
 ObjectId('5e1a1e6464b3c70d0ea1c7a3'),
 ObjectId('5e1a1e6464b3c70d0ea1c7a4'),
 ObjectId('5e1a1e6464b3c70d0ea1c7a5'),
 ObjectId('5e1a1e6464b3c70d0ea1c7a6'),
 ObjectId('5e1a1e6464b3c70d0ea1c7a7'),
 ObjectId('5e1a1e6464b3c70d0ea1c7a8'),
 ObjectId('5e1a1e6464b3c70d0ea1c7a9'),
 ObjectId('5e1a1e6464b3c70d0ea1c7aa'),
 ObjectId('5e1a1e6464b3c70d0ea1c7ab'),
 ObjectId('5e1a1e6464b3c70d0ea1c7ac'),
 ObjectId('5e1a1e6464b3c70d0ea1c7ad'),
 ObjectId('5e1a1e6464b3c70d0ea1c7ae'),
 ObjectId('5e1a1e6464b3c70d0ea1c7af'),
 ObjectId('5e1a1e6464b3c70d0ea1c7b0'),
 ObjectId('5e1a1e6464b3c70d0ea1c7b1'),
 ObjectId('5e1a1e6464b3c70d0ea1c7b2'),
 ObjectId('5e1a1e6464b3c70d0ea1c7b3'),
 ObjectId('5e1a1e6464b3c70d0ea1c7b4'),
 ObjectId('5e1a1e6464b3c70d0ea1c7b5'),
 ObjectId('5e1a1e6464b3c70d0ea1c7b6'),
 ObjectId('5e1a1e6464b3c70d0ea1c7b7'),
 ObjectId('5e1a1e6464b3c70d0ea1c7b8'),
 ObjectId('5e1a1e6464b3c7

In [37]:
db.flood_data.drop()
db.flood_data.insert(floods_data)

/Users/jeremyelam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


[ObjectId('5e1a1e6764b3c70d0ea2a6a7'),
 ObjectId('5e1a1e6764b3c70d0ea2a6a8'),
 ObjectId('5e1a1e6764b3c70d0ea2a6a9'),
 ObjectId('5e1a1e6764b3c70d0ea2a6aa'),
 ObjectId('5e1a1e6764b3c70d0ea2a6ab'),
 ObjectId('5e1a1e6764b3c70d0ea2a6ac'),
 ObjectId('5e1a1e6764b3c70d0ea2a6ad'),
 ObjectId('5e1a1e6764b3c70d0ea2a6ae'),
 ObjectId('5e1a1e6764b3c70d0ea2a6af'),
 ObjectId('5e1a1e6764b3c70d0ea2a6b0'),
 ObjectId('5e1a1e6764b3c70d0ea2a6b1'),
 ObjectId('5e1a1e6764b3c70d0ea2a6b2'),
 ObjectId('5e1a1e6764b3c70d0ea2a6b3'),
 ObjectId('5e1a1e6764b3c70d0ea2a6b4'),
 ObjectId('5e1a1e6764b3c70d0ea2a6b5'),
 ObjectId('5e1a1e6764b3c70d0ea2a6b6'),
 ObjectId('5e1a1e6764b3c70d0ea2a6b7'),
 ObjectId('5e1a1e6764b3c70d0ea2a6b8'),
 ObjectId('5e1a1e6764b3c70d0ea2a6b9'),
 ObjectId('5e1a1e6764b3c70d0ea2a6ba'),
 ObjectId('5e1a1e6764b3c70d0ea2a6bb'),
 ObjectId('5e1a1e6764b3c70d0ea2a6bc'),
 ObjectId('5e1a1e6764b3c70d0ea2a6bd'),
 ObjectId('5e1a1e6764b3c70d0ea2a6be'),
 ObjectId('5e1a1e6764b3c70d0ea2a6bf'),
 ObjectId('5e1a1e6764b3c7

In [38]:
db.earthquake_data.drop()
db.earthquake_data.insert(earthquake_data)

/Users/jeremyelam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


[ObjectId('5e1a1e6864b3c70d0ea32dc1'),
 ObjectId('5e1a1e6864b3c70d0ea32dc2'),
 ObjectId('5e1a1e6864b3c70d0ea32dc3'),
 ObjectId('5e1a1e6864b3c70d0ea32dc4'),
 ObjectId('5e1a1e6864b3c70d0ea32dc5'),
 ObjectId('5e1a1e6864b3c70d0ea32dc6'),
 ObjectId('5e1a1e6864b3c70d0ea32dc7'),
 ObjectId('5e1a1e6864b3c70d0ea32dc8'),
 ObjectId('5e1a1e6864b3c70d0ea32dc9'),
 ObjectId('5e1a1e6864b3c70d0ea32dca'),
 ObjectId('5e1a1e6864b3c70d0ea32dcb'),
 ObjectId('5e1a1e6864b3c70d0ea32dcc'),
 ObjectId('5e1a1e6864b3c70d0ea32dcd'),
 ObjectId('5e1a1e6864b3c70d0ea32dce'),
 ObjectId('5e1a1e6864b3c70d0ea32dcf'),
 ObjectId('5e1a1e6864b3c70d0ea32dd0'),
 ObjectId('5e1a1e6864b3c70d0ea32dd1'),
 ObjectId('5e1a1e6864b3c70d0ea32dd2'),
 ObjectId('5e1a1e6864b3c70d0ea32dd3'),
 ObjectId('5e1a1e6864b3c70d0ea32dd4'),
 ObjectId('5e1a1e6864b3c70d0ea32dd5'),
 ObjectId('5e1a1e6864b3c70d0ea32dd6'),
 ObjectId('5e1a1e6864b3c70d0ea32dd7'),
 ObjectId('5e1a1e6864b3c70d0ea32dd8'),
 ObjectId('5e1a1e6864b3c70d0ea32dd9'),
 ObjectId('5e1a1e6864b3c7

In [39]:
db.tornado_data.drop()
db.tornado_data.insert(tornado_data)

/Users/jeremyelam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


[ObjectId('5e1a1e6864b3c70d0ea32e82'),
 ObjectId('5e1a1e6864b3c70d0ea32e83'),
 ObjectId('5e1a1e6864b3c70d0ea32e84'),
 ObjectId('5e1a1e6864b3c70d0ea32e85'),
 ObjectId('5e1a1e6864b3c70d0ea32e86'),
 ObjectId('5e1a1e6864b3c70d0ea32e87'),
 ObjectId('5e1a1e6864b3c70d0ea32e88'),
 ObjectId('5e1a1e6864b3c70d0ea32e89'),
 ObjectId('5e1a1e6864b3c70d0ea32e8a'),
 ObjectId('5e1a1e6864b3c70d0ea32e8b'),
 ObjectId('5e1a1e6864b3c70d0ea32e8c'),
 ObjectId('5e1a1e6864b3c70d0ea32e8d'),
 ObjectId('5e1a1e6864b3c70d0ea32e8e'),
 ObjectId('5e1a1e6864b3c70d0ea32e8f'),
 ObjectId('5e1a1e6864b3c70d0ea32e90'),
 ObjectId('5e1a1e6864b3c70d0ea32e91'),
 ObjectId('5e1a1e6864b3c70d0ea32e92'),
 ObjectId('5e1a1e6864b3c70d0ea32e93'),
 ObjectId('5e1a1e6864b3c70d0ea32e94'),
 ObjectId('5e1a1e6864b3c70d0ea32e95'),
 ObjectId('5e1a1e6864b3c70d0ea32e96'),
 ObjectId('5e1a1e6864b3c70d0ea32e97'),
 ObjectId('5e1a1e6864b3c70d0ea32e98'),
 ObjectId('5e1a1e6864b3c70d0ea32e99'),
 ObjectId('5e1a1e6864b3c70d0ea32e9a'),
 ObjectId('5e1a1e6864b3c7